<a href="https://colab.research.google.com/github/MichalMaczek/praca_dyplomowa_Maczek_Myslajek/blob/main/Supervised_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sklearn imports
import sklearn.metrics
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from collections import Counter
import time

# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

In [ ]:
XY = pd.read_csv('/content/drive/MyDrive/Inż/to_classify/scaled/XY_scaled_256_150.csv', sep=',', index_col=0)

features = []
for (columnName, columnData) in XY.iteritems():
  features.append(columnName)

# X = pd.DataFrame(XY, columns= features[:-1])
# Y = pd.DataFrame(XY, columns = ["Label"])

In [ ]:
XY

In [ ]:
print(Counter(XY['Label']))

Counter({0: 1600, 1: 33})


**Split train & test data**

In [ ]:
TRAINING = [3, 4, 5, 7, 10, 11, 12, 13]
TEST = [1, 2, 6, 8, 9]
training_names = ['CTU_' + str(TRAINING[j]) + '_' for j in range(len(TRAINING))]
test_names = ['CTU_' + str(TEST[j]) + '_' for j in range(len(TEST))]

print(training_names)
print(test_names)

['CTU_3_', 'CTU_4_', 'CTU_5_', 'CTU_7_', 'CTU_10_', 'CTU_11_', 'CTU_12_', 'CTU_13_']
['CTU_1_', 'CTU_2_', 'CTU_6_', 'CTU_8_', 'CTU_9_']


In [ ]:
xy_training = XY[XY.index.str.startswith(tuple(training_names))]
x_train = pd.DataFrame(xy_training, columns= features[:-1])
y_train = pd.DataFrame(xy_training, columns = ["Label"])


xy_test = XY[XY.index.str.startswith(tuple(test_names))]
x_test = pd.DataFrame(xy_test, columns= features[:-1])
y_test = pd.DataFrame(xy_test, columns = ["Label"])

In [ ]:
x_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287,2288,2289,2290,2291,2292,2293,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
CTU_3_147.32.84.164,-0.065596,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.042620,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,0.309583,-0.092279,0.463343,0.387677,-0.102538,0.155401,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_3_147.32.84.242,0.525561,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.042620,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_3_147.32.84.59,-0.124711,-0.098425,-0.069217,-0.06182,2.543988,2.955851,2.263000,2.106956,1.270181,0.706927,0.950247,1.366574,0.874664,0.582286,0.723194,0.761591,0.652115,0.970153,0.806256,0.632271,8.190406,4.322930,6.633281,3.827303,0.682814,0.352288,1.046287,1.142485,0.344825,1.526560,1.275581,0.102405,0.774020,0.806930,0.206575,0.720179,1.004105,0.198587,0.961997,0.741208,...,6.940689,3.842152,6.555859,4.360673,6.713572,4.701210,2.014525,1.897042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_3_147.32.80.13,8.624414,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.102411,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.042620,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,0.167868,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_3_147.32.86.44,-0.124711,-0.025267,-0.069217,-0.06182,0.144082,-0.128195,-0.119708,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,0.270306,0.289566,-0.019082,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.042620,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTU_13_147.32.85.30,-0.124711,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,

In [ ]:
y_train

,Label
CTU_3_147.32.84.164,0
CTU_3_147.32.84.242,0
CTU_3_147.32.84.59,0
CTU_3_147.32.80.13,0
CTU_3_147.32.86.44,0
...,...
CTU_13_147.32.85.30,0
CTU_13_147.32.86.151,0
CTU_13_147.32.86.208,0
CTU_13_147.32.86.160,0


In [ ]:
print(Counter(y_train['Label']))

Counter({0: 888, 1: 19})


In [ ]:
x_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2264,2265,2266,2267,2268,2269,2270,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280,2281,2282,2283,2284,2285,2286,2287,2288,2289,2290,2291,2292,2293,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
CTU_1_147.32.86.194,-0.124711,-0.098425,-0.069217,-0.06182,1.125352,0.885290,0.808726,0.933708,1.127054,1.110505,1.148918,1.119939,1.198541,1.221143,1.093738,1.253283,1.127546,1.009471,1.171924,0.837354,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.04262,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_1_147.32.80.13,2.890190,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.04262,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_1_147.32.84.229,3.836041,16.874261,-0.069217,-0.06182,-0.105441,0.163319,0.285626,-0.072325,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.105924,-0.086515,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.04262,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_1_147.32.84.59,-0.124711,-0.098425,-0.069217,-0.06182,5.939183,6.020826,6.231441,5.404659,5.822697,5.994347,5.897979,5.971322,5.509647,5.224277,4.123970,3.501020,3.331308,3.180370,3.277370,3.014390,0.723227,0.477369,0.108272,0.337931,-0.054838,-0.052915,0.174781,-0.04262,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,3.821986,13.066081,6.555859,10.485049,10.119824,7.446210,3.641257,10.476569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTU_1_147.32.84.138,-0.124711,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.078710,-0.054838,-0.052915,-0.043095,-0.04262,-0.047921,-0.052473,-0.057355,-0.059952,-0.065982,-0.067854,-0.070271,-0.067484,-0.062808,-0.054371,-0.047692,-0.047135,...,-0.076392,-0.082924,-0.092279,-0.093419,-0.098931,-0.102538,-0.076989,-0.082849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTU_9_147.32.86.166,-0.124711,-0.098425,-0.069217,-0.06182,-0.125066,-0.128195,-0.125186,-0.121667,-0.122548,-0.116591,-0.113869,-0.110524,-0.110576,-0.108666,-0.105728,-0.106597,-0.102981,-0.099851,-0.099344,-0.098666,-0.101636,-0.097255,-0.086504,-0.07

In [ ]:
y_test

,Label
CTU_1_147.32.86.194,0
CTU_1_147.32.80.13,0
CTU_1_147.32.84.229,0
CTU_1_147.32.84.59,0
CTU_1_147.32.84.138,0
...,...
CTU_9_147.32.86.166,0
CTU_9_147.32.85.6,0
CTU_9_147.32.86.155,0
CTU_9_147.32.85.40,0


In [ ]:
print(Counter(y_test['Label']))

Counter({0: 712, 1: 14})


# SAMPLING


OVERSAMPLING minority

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# summarize class distribution
print(Counter(y_train['Label']))
print(y_train[y_train['Label']==1].index)
# define oversampling strategy
over = RandomOverSampler(sampling_strategy=0.1)
# fit and apply the transform
x_train_over, y_train_over = over.fit_resample(x_train, y_train)
# summarize class distribution
print(Counter(y_train_over['Label']))
print(y_train_over[y_train_over['Label']==1].index)

# testy w poszukiwaniu najlepszych parametrów

# **SVM**

In [ ]:
from sklearn.svm import LinearSVC,SVC

## svc
> normal

In [ ]:
svm_classifier = SVC(kernel='linear', C=0.001) # Linear Kernel

In [ ]:
svm_classifier.fit(x_train, y_train)

In [ ]:
svm_classifier_predictions = svm_classifier.predict(x_test)

svm_classifier_cls = classification_report(y_test, svm_classifier_predictions)
svm_classifier_con = confusion_matrix(y_test, svm_classifier_predictions)  
print(svm_classifier_cls)
print(svm_classifier_con)

## grid search


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
svm_classifier = SVC() # Linear Kernel

parameter_space = {
    'kernel': ['linear', 'poly', 'sigmoid'],
    'C': [1000, 500, 100, 50, 10, 1.0, 0.1, 0.01, 0.001]
}


grid_svm_classifier = GridSearchCV(svm_classifier, parameter_space, n_jobs=-1, scoring='f1', cv=2)
# grid_mlp_classifier = GridSearchCV(mlp_classifier, parameter_space, n_jobs=-1, cv=5)

grid_svm_classifier.fit(x_train_under, y_train_under)

In [ ]:
# Best paramete set
print('Best parameters found:\n', grid_svm_classifier.best_params_, ': ', grid_svm_classifier.best_score_)

# All results
means = grid_svm_classifier.cv_results_['mean_test_score']
stds = grid_svm_classifier.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_svm_classifier.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

# **KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## normal

### k = 3

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=3)

  knn3_classifier.fit(x_train, y_train)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)

  print('3-neighbors')
  print(cls_knn3)
  print(con_knn3)

### k = 5


In [ ]:
# no threshold
with tf.device('/device:GPU:0'):
  knn5_classifier = KNeighborsClassifier(n_neighbors=5)

  knn5_classifier.fit(x_train, y_train)
  y_pred = knn5_classifier.predict(x_test)
  cls_knn5 = classification_report(y_test, y_pred)
  con_knn5 = confusion_matrix(y_test, y_pred)
  
  print('5-neighbors')
  print(cls_knn5)
  print(con_knn5)

### k = 7

In [ ]:
# no threshold
with tf.device('/device:GPU:0'):
  knn7_classifier = KNeighborsClassifier(n_neighbors=7)
  
  knn7_classifier.fit(x_train, y_train)
  y_pred = knn7_classifier.predict(x_test)
  cls_knn7 = classification_report(y_test, y_pred)
  con_knn7 = confusion_matrix(y_test, y_pred)

  print('7-neighbors')
  print(cls_knn7)
  print(con_knn7)

### k = 9

In [ ]:
# no threshold
with tf.device('/device:GPU:0'):
  knn9_classifier = KNeighborsClassifier(n_neighbors=9)
  
  knn9_classifier.fit(x_train, y_train)
  y_pred = knn9_classifier.predict(x_test)
  cls_knn9 = classification_report(y_test, y_pred)
  con_knn9 = confusion_matrix(y_test, y_pred)
  
  print('9-neighbors')
  print(cls_knn9)
  print(con_knn9)

## oversampled

## 10%

### 3

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=3)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('3-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 5


In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=5)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('5-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 7

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=7)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('7-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 9

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=9)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('9-neighbors')
  print(cls_knn3)
  print(con_knn3)

## 5%

### 3

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=3)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('3-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 5


In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=5)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('5-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 7

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=7)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('7-neighbors')
  print(cls_knn3)
  print(con_knn3)
  print(Counter(y_train_under))

### 9

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=9)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('9-neighbors')
  print(cls_knn3)
  print(con_knn3)

## 15%

### 3

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=3)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('3-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 5


In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=5)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('5-neighbors')
  print(cls_knn3)
  print(con_knn3)

In [ ]:
result = np.where(y_pred == 1)
result

(array([146, 288, 367, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720]),)

In [ ]:
result1 = np.where(y_test)

In [ ]:
result1 = np.where([y_test['Label']==1]) 
result1

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([146, 288, 367, 463, 711, 712, 713, 714, 715, 716, 717, 718, 719,
        720]))

### 7

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=7)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('7-neighbors')
  print(cls_knn3)
  print(con_knn3)
  print(Counter(y_train_under))

### 9

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=9)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred)
  con_knn3 = confusion_matrix(y_test, y_pred)
  
  print('9-neighbors')
  print(cls_knn3)
  print(con_knn3)

# testy zależności od no_of_bins

In [ ]:
XY = pd.read_csv('/content/drive/MyDrive/Inż/to_classify/scaled/XY_scaled_1024_150.csv', sep=',', index_col=0)

features = []
for (columnName, columnData) in XY.iteritems():
  features.append(columnName)

# X = pd.DataFrame(XY, columns= features[:-1])
# Y = pd.DataFrame(XY, columns = ["Label"])

TRAINING = [3, 4, 5, 7, 10, 11, 12, 13]
TEST = [1, 2, 6, 8, 9]
training_names = ['CTU_' + str(TRAINING[j]) + '_' for j in range(len(TRAINING))]
test_names = ['CTU_' + str(TEST[j]) + '_' for j in range(len(TEST))]

print(training_names)
print(test_names)

xy_training = XY[XY.index.str.startswith(tuple(training_names))]
x_train = pd.DataFrame(xy_training, columns= features[:-1])
y_train = pd.DataFrame(xy_training, columns = ["Label"])


xy_test = XY[XY.index.str.startswith(tuple(test_names))]
x_test = pd.DataFrame(xy_test, columns= features[:-1])
y_test = pd.DataFrame(xy_test, columns = ["Label"])

print(x_train.shape)

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# summarize class distribution
print(Counter(y_train['Label']))
# print(y_train[y_train['Label']==1].index)
# define oversampling strategy
over = RandomOverSampler(sampling_strategy=0.1)
# fit and apply the transform
x_train_over, y_train_over = over.fit_resample(x_train, y_train)
# summarize class distribution
print(Counter(y_train_over['Label']))
# print(y_train_over[y_train_over['Label']==1].index)

In [ ]:
XY

# **SVM**

In [ ]:
from sklearn.svm import LinearSVC,SVC

## svc
> normal

In [ ]:
svm_classifier = SVC(kernel='linear') # Linear Kernel

In [ ]:
svm_classifier.fit(x_train, y_train)

In [ ]:
svm_classifier_predictions = svm_classifier.predict(x_test)

svm_classifier_cls = classification_report(y_test, svm_classifier_predictions)
svm_classifier_con = confusion_matrix(y_test, svm_classifier_predictions)  
print(svm_classifier_cls)
print(svm_classifier_con)

In [ ]:
# to change
svm_1024_rec = sklearn.metrics.recall_score(y_test, svm_classifier_predictions, pos_label=1)
svm_1024_prec = sklearn.metrics.precision_score(y_test, svm_classifier_predictions, pos_label=1)
svm_1024_f1 = sklearn.metrics.f1_score(y_test, svm_classifier_predictions, pos_label=1)
print(svm_1024_rec)
print(svm_1024_prec)
print(svm_1024_f1)

# **KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## oversampled 10%

### 3

In [ ]:
with tf.device('/device:GPU:0'):
  knn3_classifier = KNeighborsClassifier(n_neighbors=3)

  knn3_classifier.fit(x_train_over, y_train_over)
  y_pred_knn3 = knn3_classifier.predict(x_test)
  cls_knn3 = classification_report(y_test, y_pred_knn3)
  con_knn3 = confusion_matrix(y_test, y_pred_knn3)
  
  print('3-neighbors')
  print(cls_knn3)
  print(con_knn3)

### 5


In [ ]:
with tf.device('/device:GPU:0'):
  knn5_classifier = KNeighborsClassifier(n_neighbors=5)

  knn5_classifier.fit(x_train_over, y_train_over)
  y_pred_knn5 = knn5_classifier.predict(x_test)
  cls_knn5 = classification_report(y_test, y_pred_knn5)
  con_knn5 = confusion_matrix(y_test, y_pred_knn5)
  
  print('5-neighbors')
  print(cls_knn5)
  print(con_knn5)

### 7

In [ ]:
with tf.device('/device:GPU:0'):
  knn7_classifier = KNeighborsClassifier(n_neighbors=7)

  knn7_classifier.fit(x_train_over, y_train_over)
  y_pred_knn7 = knn7_classifier.predict(x_test)
  cls_knn7 = classification_report(y_test, y_pred_knn7)
  con_knn7 = confusion_matrix(y_test, y_pred_knn7)
  
  print('7-neighbors')
  print(cls_knn7)
  print(con_knn7)

### 9

In [ ]:
with tf.device('/device:GPU:0'):
  knn9_classifier = KNeighborsClassifier(n_neighbors=9)

  knn9_classifier.fit(x_train_over, y_train_over)
  y_pred_knn9 = knn9_classifier.predict(x_test)
  cls_knn9 = classification_report(y_test, y_pred_knn9)
  con_knn9 = confusion_matrix(y_test, y_pred_knn9)
  
  print('9-neighbors')
  print(cls_knn9)
  print(con_knn9)

###  

In [ ]:
# to change
knn_1024_rec = sklearn.metrics.recall_score(y_test, y_pred_knn9, pos_label=1)
knn_1024_prec = sklearn.metrics.precision_score(y_test, y_pred_knn9, pos_label=1)
knn_1024_f1 = sklearn.metrics.f1_score(y_test, y_pred_knn9, pos_label=1)
print(knn_1024_rec)
print(knn_1024_prec)
print(knn_1024_f1)

# **MLP**

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
sign_size = 2304

In [ ]:
import warnings
warnings.filterwarnings('default')

# sklearn
rec = []
prec = []
f1 = []

# initial hyperparameters
# dense_1_size = 192
dense_1_size = int(sign_size/2)
print(dense_1_size)
# learning_rate = 0.05
epochs = 70

# the model
model_16_relu_sigmoid_sklearn = MLPClassifier(
    hidden_layer_sizes=(dense_1_size),
    activation="relu",
    solver="adam",
    batch_size=128
)

for i in range(25):
  model_16_relu_sigmoid_sklearn.fit(x_train, y_train)

  y_pred_mlp_sklearn = model_16_relu_sigmoid_sklearn.predict(x_test)
  
  rec.append(sklearn.metrics.recall_score(y_test, y_pred_mlp_sklearn, pos_label=1))
  prec.append(sklearn.metrics.precision_score(y_test, y_pred_mlp_sklearn, pos_label=1))
  f1.append(sklearn.metrics.f1_score(y_test, y_pred_mlp_sklearn, pos_label=1))

print('Recalls: ' + str(rec))
print('Precisions: ' + str(prec))
print('F1: ' + str(f1))

vals_rec, counts_rec = np.unique(rec, return_counts=True)
index_rec = np.argmax(counts_rec)

vals_prec, counts_prec = np.unique(prec, return_counts=True)
index_prec = np.argmax(counts_prec)

vals_f1, counts_f1 = np.unique(f1, return_counts=True)
index_f1 = np.argmax(counts_f1)


mlp_rec_avg = round(sum(rec) / len(rec), 4)
mlp_rec_min = min(rec)
mlp_rec_max = max(rec)
mlp_rec_mode = vals_rec[index_rec]

mlp_prec_avg = round(sum(prec) / len(prec), 4)
mlp_prec_min = min(prec)
mlp_prec_max = max(prec)
mlp_prec_mode = vals_prec[index_prec]

mlp_f1_avg = round(sum(f1) / len(f1), 4)
mlp_f1_min = min(f1)
mlp_f1_max = max(f1)
mlp_f1_mode = vals_f1[index_f1]


print('----MLP-best-'+str(sign_size/9)+'----')

print('Recall avg: ' + str(mlp_rec_avg))
print('Recall min: ' + str(mlp_rec_min))
print('Recall max: ' + str(mlp_rec_max))
print('Recall mode: ' + str(mlp_rec_mode))

print('Precision avg: ' + str(mlp_prec_avg))
print('Precision min: ' + str(mlp_prec_min))
print('Precision max: ' + str(mlp_prec_max))
print('Precision mode: ' + str(mlp_prec_mode))

print('F1 score avg: ' + str(mlp_f1_avg))
print('F1 score min: ' + str(mlp_f1_min))
print('F1 score max: ' + str(mlp_f1_max))
print('F1 score mode: ' + str(mlp_f1_mode))

In [ ]:
# to change
mlp_256_rec = mlp_rec_avg
mlp_256_prec = mlp_prec_avg
mlp_256_f1 = mlp_f1_avg
print(mlp_256_rec)
print(mlp_256_prec)
print(mlp_256_f1)

# **WYKRESY**

In [ ]:
print('svm_1024_rec: ', svm_1024_rec)
print('svm_1024_prec: ', svm_1024_prec)
print('svm_1024_f1: ', svm_1024_f1)

# 5 neighbors
print('knn_1024_rec: ', knn_1024_rec)
print('knn_1024_prec: ', knn_1024_prec)
print('knn_1024_f1: ', knn_1024_f1)

print('mlp_1024_rec: ', mlp_1024_rec)
print('mlp_1024_prec: ', mlp_1024_prec)
print('mlp_1024_f1: ', mlp_1024_f1)

In [ ]:
print('svm_512_rec: ', svm_512_rec)
print('svm_512_prec: ', svm_512_prec)
print('svm_512_f1: ', svm_512_f1)

# 5 neighbors
print('knn_512_rec: ', knn_512_rec)
print('knn_512_prec: ', knn_512_prec)
print('knn_512_f1: ', knn_512_f1)

print('mlp_512_rec: ', mlp_512_rec)
print('mlp_512_prec: ', mlp_512_prec)
print('mlp_512_f1: ', mlp_512_f1)

In [ ]:
print('svm_256_rec: ', svm_256_rec)
print('svm_256_prec: ', svm_256_prec)
print('svm_256_f1: ', svm_256_f1)

# 5 neighbors
print('knn_256_rec: ', knn_256_rec)
print('knn_256_prec: ', knn_256_prec)
print('knn_256_f1: ', knn_256_f1)

print('mlp_256_rec: ', mlp_256_rec)
print('mlp_256_prec: ', mlp_256_prec)
print('mlp_256_f1: ', mlp_256_f1)

In [ ]:
print('svm_128_rec: ', svm_128_rec)
print('svm_128_prec: ', svm_128_prec)
print('svm_128_f1: ', svm_128_f1)

# 5 neighbors
print('knn_128_rec: ', knn_128_rec)
print('knn_128_prec: ', knn_128_prec)
print('knn_128_f1: ', knn_128_f1)

print('mlp_128_rec: ', mlp_128_rec)
print('mlp_128_prec: ', mlp_128_prec)
print('mlp_128_f1: ', mlp_128_f1)

In [ ]:
print('svm_64_rec: ', svm_64_rec)
print('svm_64_prec: ', svm_64_prec)
print('svm_64_f1: ', svm_64_f1)

# 7 neighbors
print('knn_64_rec: ', knn_64_rec)
print('knn_64_prec: ', knn_64_prec)
print('knn_64_f1: ', knn_64_f1)

print('mlp_64_rec: ', mlp_64_rec)
print('mlp_64_prec: ', mlp_64_prec)
print('mlp_64_f1: ', mlp_64_f1)

In [ ]:
print('svm_32_rec: ', svm_32_rec)
print('svm_32_prec: ', svm_32_prec)
print('svm_32_f1: ', svm_32_f1)

# 7 neighbors
print('knn_32_rec: ', knn_32_rec)
print('knn_32_prec: ', knn_32_prec)
print('knn_32_f1: ', knn_32_f1)

print('mlp_32_rec: ', mlp_32_rec)
print('mlp_32_prec: ', mlp_32_prec)
print('mlp_32_f1: ', mlp_32_f1)

In [ ]:
svm = {
    "rec":[svm_32_rec, svm_64_rec, svm_128_rec, svm_256_rec, svm_512_rec, svm_1024_rec], 
    "prec":[svm_32_prec, svm_64_prec, svm_128_prec, svm_256_prec, svm_512_prec, svm_1024_prec],
    "f1":[svm_32_f1, svm_64_f1, svm_128_f1, svm_256_f1, svm_512_f1, svm_1024_f1]
}
knn = {
    "rec":[knn_32_rec, knn_64_rec, knn_128_rec, knn_256_rec, knn_512_rec, knn_1024_rec], 
    "prec":[knn_32_prec, knn_64_prec, knn_128_prec, knn_256_prec, knn_512_prec, knn_1024_prec],
    "f1":[knn_32_f1, knn_64_f1, knn_128_f1, knn_256_f1, knn_512_f1, knn_1024_f1]
}
mlp = {
    "rec":[mlp_32_rec, mlp_64_rec, mlp_128_rec, mlp_256_rec, mlp_512_rec, mlp_1024_rec], 
    "prec":[mlp_32_prec, mlp_64_prec, mlp_128_prec, mlp_256_prec, mlp_512_prec, mlp_1024_prec],
    "f1":[mlp_32_f1, mlp_64_f1, mlp_128_f1, mlp_256_f1, mlp_512_f1, mlp_1024_f1]
}

x = ['32', '64', '128', '256', '512', '1024']

print(svm)
print(knn)
print(mlp)
print(x)

In [ ]:
fig = plt.figure(figsize=(9,6), dpi=80)
plt.plot(x, svm['prec'], color='orange', marker='v', linestyle='-.')
plt.plot(x, knn['prec'], color='green', marker='s', linestyle='dashed')
plt.plot(x, mlp['prec'], color='red', marker='<', linestyle='-')
plt.ylim([0, 1.05])
plt.grid(linestyle = '--', linewidth = 0.5)
plt.xlabel('Liczba binów', fontsize=11)
# plt.title("Precyzja", fontsize=20)
plt.legend(["SVM", "KNN", "MLP"], loc=4)
fig.savefig('wyniki_prec.png')

In [ ]:
fig = plt.figure(figsize=(9,6), dpi=80)
plt.plot(x, svm['rec'], color='orange', marker='v', linestyle='-.')
plt.plot(x, knn['rec'], color='green', marker='s', linestyle='dashed')
plt.plot(x, mlp['rec'], color='red', marker='<', linestyle='-')
plt.ylim([0, 1.05])
plt.grid(linestyle = '--', linewidth = 0.5)
plt.xlabel('Liczba binów', fontsize=11)
# plt.title("Czułość", fontsize=20)
plt.legend(["SVM", "KNN", "MLP"], loc=4)
fig.savefig('wyniki_rec.png')

In [ ]:
fig = plt.figure(figsize=(9,6), dpi=80)
plt.plot(x, svm['f1'], color='orange', marker='v', linestyle='-.')
plt.plot(x, knn['f1'], color='green', marker='s', linestyle='dashed')
plt.plot(x, mlp['f1'], color='red', marker='<', linestyle='-')
plt.ylim([0, 1.05])
plt.grid(linestyle = '--', linewidth = 0.5)
plt.xlabel('Liczba binów', fontsize=11)
# plt.title("Miara F1", fontsize=20)
plt.legend(["SVM", "KNN", "MLP"], loc=4)
fig.savefig('wyniki_f1.png')